In [1]:
import dropbox
import os
import mp
from tqdm import tqdm

access_token = ''
dbx = dropbox.Dropbox(access_token)

In [2]:
from bs4 import BeautifulSoup

with open('Diff Room Scripts - Dropbox.html') as fopen:
    html = fopen.read()
    
bs = BeautifulSoup(html)
hrefs = bs.select('a[href*=".TextGrid"]')
hrefs_text = [href['href'] for href in hrefs]

In [3]:
len(hrefs_text)

972

In [4]:
local_directory = 'TEXT'
os.system(f'mkdir {local_directory}')

256

In [5]:
def loop(hrefs):
    hrefs = hrefs[0]
    for i in tqdm(range(len(hrefs))):
        href = hrefs[i]
        while True:
            try:
                name = dbx.sharing_get_shared_link_metadata(href).name
                break
            except Exception as e:
                print(e)
        d = f'{local_directory}/{name}'
        if os.path.exists(d):
            print(f'{d} exists, skip')
            continue
        while True:
            try:
                dbx.sharing_get_shared_link_file_to_file(d, href)
                break
            except Exception as e:
                print(e)

In [6]:
loop((hrefs_text[:1],))

100%|██████████| 1/1 [00:00<00:00,  2.07it/s]

TEXT/sur_0007_1014_phnd_cs-chn.TextGrid exists, skip


In [7]:
# mp.multiprocessing(hrefs_text, loop, cores = 6, returned = False)

In [8]:
with open('Diff Room Audio - Dropbox.html') as fopen:
    html = fopen.read()
    
bs = BeautifulSoup(html)
hrefs = bs.select('a[href*=".wav"]')
hrefs = [href['href'] for href in hrefs]

In [9]:
from collections import defaultdict
from glob import glob
import os

speakers = {}
files = glob('TEXT/*.TextGrid')
for path in files:
    new_path = os.path.split(path)[1].replace(' ', '-').replace('.TextGrid', '')
    f = os.path.split(path)[1].replace('.TextGrid', '')
    speakers[f] = path

In [10]:
from tqdm import tqdm

directory = 'part4-diff-room'

os.system(f'mkdir {directory}')
os.system(f'mkdir {directory}/wav')
os.system(f'mkdir {directory}/text')

256

In [21]:
import soundfile as sf
import textgrid

In [22]:
def split(path):
    new_path = os.path.split(path)[1].replace(' ', '-').replace('.wav', '')
    f = os.path.split(path)[1].replace('.wav', '')
    y, sr = sf.read(path)
    index = 0
    path_textgrid = speakers[f]
    tg = textgrid.TextGrid.fromFile(path_textgrid)
    for i in range(len(tg)):
        for k in range(len(tg[i])):
            try:
                splitted = y[int(sr * tg[i][k].minTime): int(sr * tg[i][k].maxTime)]
                sf.write(f'{directory}/wav/{new_path}-{i}-{k}.wav', splitted, sr)
                with open(f'{directory}/text/{new_path}-{i}-{k}.wav.txt', 'w') as fopen:
                    fopen.write(tg[i][k].mark)
            except Exception as e:
                print('nested exception', e)

In [23]:
split('WAVE/sur_0007_1014_phnd_cs-chn.wav')

In [26]:
wavs = glob(f'{directory}/wav/*.wav')
wavs[0]

'part4-diff-room/wav/sur_0007_1014_phnd_cs-chn-0-427.wav'

In [27]:
import IPython.display as ipd
ipd.Audio(wavs[0])

In [30]:
!cat {wavs[0].replace('/wav/', '/text/') + '.txt'}

like photography [ah]

In [31]:
local_directory = 'WAVE'
os.system(f'mkdir {local_directory}')

def loop(hrefs):
    hrefs = hrefs[0]
    for i in tqdm(range(len(hrefs))):
        href = hrefs[i]
        while True:
            try:
                name = dbx.sharing_get_shared_link_metadata(href).name
                break
            except:
                pass
        d = f'{local_directory}/{name}'
        if not os.path.exists(d):
            while True:
                try:
                    dbx.sharing_get_shared_link_file_to_file(d, href)
                    break
                except Exception as e:
                    print(e)
        
        print(d, name)
        try:
            split(d)
        except:
            pass
        os.remove(d)

In [32]:
loop((hrefs[:1],))

  0%|          | 0/1 [00:00<?, ?it/s]

WAVE/sur_0007_1014_phnd_cs-chn.wav sur_0007_1014_phnd_cs-chn.wav


100%|██████████| 1/1 [00:09<00:00,  9.68s/it]


In [46]:
mp.multiprocessing(hrefs, loop, cores = 4, returned = False)

In [47]:
# !tar --use-compress-program="pigz -0 -k " -cf part4-diff-room.tar.gz part4-diff-room 